# Challenge 2

In this challenge we will continue working with the `Pokemon` dataset. We will attempt solving a slightly more complex problem in which we will practice the iterative data analysis process you leaned in [this video](https://www.youtube.com/watch?v=xOomNicqbkk).

The problem statement is as follows:

**You are at a Pokemon black market planning to buy a Pokemon for battle. All Pokemon are sold at the same price and you can only afford to buy one. You cannot choose which specific Pokemon to buy. However, you can specify the type of the Pokemon - one type that exists in either `Type 1` or `Type 2`. Which type should you choose in order to maximize your chance of receiving a good Pokemon?**

To remind you about the 3 steps of iterative data analysis, they are:

1. Setting Expectations
1. Collecting Information
1. Reacting to Data / Revising Expectations

Following the iterative process, we'll guide you in completing the challenge.


## Problem Solving Iteration 1

In this iteration we'll analyze the problem and identify the breakthrough. The original question statement is kind of vague because we don't know what a *good pokemon* really means as represented in the data. We'll start by understanding the dataset and see if we can find some insights.

In [1]:
# Import libraries
import numpy as np
import pandas as pd

In [2]:
# Importing the dataset
pokemon = pd.read_csv(r"G:\My Drive\Ironhack\lab-dataframe-calculation-and-transformation\data\Pokemon.csv")
pokemon.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


From the data it seems whether a pokemon is good depends on its abilities as represented in the fields of `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total`. We are not sure about `Generation` and `Legendary` because they are not necessarily the decisive factors of the pokemon abilities.

But `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total` are a lot of fields! If we look at them all at once it's very complicated. This isn't Mission Impossible but it's ideal that we tackle this kind of problem after we learn Machine Learning (which you will do in Module 3). For now, is there a way to consolidate the fields we need to look into?

Fortunately there seems to be a way. It appears the `Total` field is computed based on the other 6 fields. But we need to prove our theory. If we can approve there is a formula to compute `Total` based on the other 6 abilities, we only need to look into `Total`.

We have the following expectation now:

#### The `Total` field is computed based on `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, and `Speed`.

We need to collect the following information:

* **What is the formula to compute `Total`?**
* **Does the formula work for all pokemon?**

In the cell below, make a hypothesis on how `Total` is computed and test your hypothesis.

In [3]:
# your code here
pokemon["total_test"] = pokemon["HP"] + pokemon["Attack"] + pokemon["Sp. Atk"] + pokemon["Defense"] + pokemon["Sp. Def"] + pokemon["Speed"]
pokemon
comparison = pokemon["total_test"] == pokemon["Total"]
comparison.unique()

# --> Total is correct

array([ True])

## Problem Solving Iteration 2

Now that we have consolidated the abilities fields, we can update the problem statement. The new problem statement is:

### Which pokemon type is most likely to have the highest `Total` value?

In the updated problem statement, we assume there is a certain relationship between the `Total` and the pokemon type. But we have two *type* fields (`Type 1` and `Type 2`) that have string values. In data analysis, string fields have to be transformed to numerical format in order to be analyzed. 

In addition, keep in mind that `Type 1` always has a value but `Type 2` is sometimes empty (having the `NaN` value). Also, the pokemon type we choose may be either in `Type 1` or `Type 2`.

Now our expectation is:

#### `Type 1` and `Type 2` string variables need to be converted to numerical variables in order to identify the relationship between `Total` and the pokemon type.

The information we need to collect is:

#### How to convert two string variables to numerical?

Let's address the first question first. You can use a method called **One Hot Encoding** which is frequently used in machine learning to encode categorical string variables to numerical. The idea is to gather all the possible string values in a categorical field and create a numerical field for each unique string value. Each of those numerical fields uses `1` and `0` to indicate whether the data record has the corresponding categorical value. A detailed explanation of One Hot Encoding can be found in [this article](https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f). You will formally learn it in Module 3.

For instance, if a pokemon has `Type 1` as `Poison` and `Type 2` as `Fire`, then its `Poison` and `Fire` fields are `1` whereas all other fields are `0`. If a pokemon has `Type 1` as `Water` and `Type 2` as `NaN`, then its `Water` field is `1` whereas all other fields are `0`.

#### In the next cell, use One Hot Encoding to encode `Type 1` and `Type 2`. Use the pokemon type values as the names of the numerical fields you create.

The new numerical variables you create should look like below:

![One Hot Encoding](../images/one-hot-encoding.png)

In [4]:
# your code here
pokemon_names = pokemon["Name"].tolist()
pokemon_types = pokemon["Type 1"].unique()
np.append(pokemon_types, "Total")

df = pd.DataFrame(0, index=pokemon_names, columns=pokemon_types)


# for pokemon_name in pokemon_names:
#     df.loc[pokemon_name, pokemon[pokemon["Name"] == pokemon_name]["Type 1"].iloc[0]] = 1
#     if pokemon[pokemon["Name"] == pokemon_name]["Type 2"].iloc[0] is False:
#         df.loc[pokemon_name, pokemon[pokemon["Name"] == pokemon_name]["Type 2"].iloc[0]] = 1
# df

# Loop through each pokemon
for pokemon_name in pokemon_names:
    # Get the row for this Pokémon
    row = pokemon[pokemon["Name"] == pokemon_name].iloc[0]
    # Set the Type 1 value to 1
    df.at[pokemon_name, row["Type 1"]] = 1
    
    # Set the Type 2 value to 1 if it exists
    if pd.notna(row["Type 2"]):
        df.at[pokemon_name, row["Type 2"]] = 1

    df.at[pokemon_name, "Total"] = row["Total"]
df

,Grass,Fire,Water,Bug,Normal,Poison,Electric,Ground,Fairy,Fighting,Psychic,Rock,Ghost,Ice,Dragon,Dark,Steel,Flying,Total
Bulbasaur,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,318.0
Ivysaur,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,405.0
Venusaur,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,525.0
VenusaurMega Venusaur,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,625.0
Charmander,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,309.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Diancie,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,600.0
DiancieMega Diancie,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,700.0
HoopaHoopa Confined,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,600.0
HoopaHoopa Unbound,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,680.0


## Problem Solving Iteration 3

Now we have encoded the pokemon types, we will identify the relationship between `Total` and the encoded fields. Our expectation is:

#### There are relationships between `Total` and the encoded pokemon type variables and we need to identify the correlations.

The information we need to collect is:

#### How to identify the relationship between `Total` and the encoded pokemon type fields?

There are multiple ways to answer this question. The easiest way is to use correlation. In the cell below, calculate the correlation of `Total` to each of the encoded fields. Rank the correlations and identify the #1 pokemon type that is most likely to have the highest `Total`.

In [5]:
df_corre = df.corr()
df_corre[["Total"]].sort_values(by="Total", ascending=False)

,Total
Total,1.000000
Dragon,0.229705
Psychic,0.124688
Steel,0.109703
Fire,0.078726
Fighting,0.077786
Ice,0.060248
Flying,0.059383
Dark,0.056154
Rock,0.032731


# Bonus Question

Say now you can choose both `Type 1` and `Type 2` of the pokemon. In order to receive the best pokemon, which types will you choose?

In [6]:
# your code here
pokemon['Combo Type'] = np.where(pd.notna(pokemon['Type 2']),
                            pokemon['Type 1'] + '-' + pokemon['Type 2'], 
                            pokemon['Type 1'])
pokemon_and_type = pokemon[["Total", "Combo Type"]]

pokemon_and_type.groupby(["Combo Type"])[["Total"]].mean().sort_values("Total", ascending=False)

,Total
Combo Type,
Ground-Fire,770.000000
Dragon-Ice,686.666667
Steel-Dragon,680.000000
Ghost-Dragon,680.000000
Dragon-Fire,680.000000
...,...
Poison-Bug,330.000000
Poison-Water,320.000000
Bug,289.705882


In [7]:
pokemon_types2 = pokemon["Combo Type"].unique().tolist()

df2 = pd.DataFrame(0, index=pokemon_names, columns=pokemon_types2)


# for pokemon_name in pokemon_names:
#     df.loc[pokemon_name, pokemon[pokemon["Name"] == pokemon_name]["Type 1"].iloc[0]] = 1
#     if pokemon[pokemon["Name"] == pokemon_name]["Type 2"].iloc[0] is False:
#         df.loc[pokemon_name, pokemon[pokemon["Name"] == pokemon_name]["Type 2"].iloc[0]] = 1
# df

# Loop through each pokemon
for pokemon_name in pokemon_names:
    # Get the row for this Pokémon
    row = pokemon[pokemon["Name"] == pokemon_name].iloc[0]
    # Set the Type 1 value to 1
    df2.at[pokemon_name, row["Combo Type"]] = 1


    df2.at[pokemon_name, "Total"] = row["Total"]
df2

,Grass-Poison,Fire,Fire-Flying,Fire-Dragon,Water,Bug,Bug-Flying,Bug-Poison,Normal-Flying,Normal,...,Rock-Ice,Fighting-Flying,Electric-Fairy,Rock-Fairy,Ghost-Grass,Flying-Dragon,Psychic-Ghost,Psychic-Dark,Fire-Water,Total
Bulbasaur,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,318.0
Ivysaur,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,405.0
Venusaur,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,525.0
VenusaurMega Venusaur,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,625.0
Charmander,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,309.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Diancie,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,600.0
DiancieMega Diancie,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,700.0
HoopaHoopa Confined,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,600.0
HoopaHoopa Unbound,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,680.0


In [8]:
df_corre2 = df2.corr()
df_corre2[["Total"]].sort_values(by="Total", ascending=False)

,Total
Total,1.000000
Dragon-Flying,0.149777
Dragon-Ice,0.128737
Dragon-Psychic,0.127066
Psychic-Fighting,0.104173
...,...
Poison,-0.064380
Normal-Fairy,-0.069525
Bug-Poison,-0.089742
Normal-Flying,-0.092626
